In [1]:
using SparseArrays

In [32]:
cor_files = [f for f in readdir("tssp/") if f[end-3:end] == ".cor"]

12-element Array{String,1}:
 "4node.cor"   
 "AIRL.cor"    
 "LandS.cor"   
 "assets.cor"  
 "chem.cor"    
 "env.cor"     
 "fxm.cor"     
 "phone.cor"   
 "pltexpA2.cor"
 "stocfor1.cor"
 "stocfor2.cor"
 "stormG2.cor" 

In [31]:
tim_files = [f for f in readdir("tssp/") if f[end-3:end] == ".tim"]

12-element Array{String,1}:
 "4node.tim"   
 "AIRL.tim"    
 "LandS.tim"   
 "assets.tim"  
 "chem.tim"    
 "env.tim"     
 "fxm2.tim"    
 "phone.tim"   
 "pltexpA2.tim"
 "stocfor1.tim"
 "stocfor2.tim"
 "stormG2.tim" 

In [34]:
sto_files = [f for f in readdir("tssp/") if occursin(".sto", f)];

In [41]:
# Files for which
for f in sto_files
    f_ = String(split(f, '.')[1])
    if !in(f_*".cor", cor_files)
        println(f)
    end
end

LandS_blocks.sto
fxm2_16.sto
fxm2_6.sto
fxmev.sto
pltexpA2_16.sto
pltexpA2_6.sto
stormG2_1000.sto
stormG2_125.sto
stormG2_27.sto
stormG2_8.sto


# AIRL tests

## Parsing `.cor` file

In [168]:
include("src/core_parser.jl")

parsemps_bounds! (generic function with 1 method)

In [169]:
d, row2idx, col2idx, I, J, V, objI, objV, rhsI, rhsV, senses = parse_cor_file("tssp/AIRL.cor")

(Dict{String,Any}("name"=>"AIRL","nrows"=>8,"range"=>"","rhs"=>"RIGHT","ncols"=>12,"bound"=>""), Dict{Any,Any}("HOURS2"=>2,"OBJ"=>0,"AVAIL12"=>4,"HOURS1"=>1,"AVAIL21"=>5,"DEMAND1"=>7,"AVAIL11"=>3,"DEMAND2"=>8,"AVAIL22"=>6), Dict{Any,Any}("YMINUS2"=>12,"YPLUS2"=>10,"X121"=>6,"X21"=>3,"X212"=>7,"X11"=>1,"X22"=>4,"RIGHT"=>0,"YMINUS1"=>11,"YPLUS1"=>9…), [1, 3, 7, 1, 4, 8, 2, 5, 7, 2  …  5, 7, 8, 6, 8, 7, 7, 8, 7, 8], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  7, 7, 7, 8, 8, 8, 9, 10, 11, 12], [24.0, -24.0, 50.0, 14.0, -14.0, 75.0, 49.0, -49.0, 20.0, 29.0  …  36.0, -14.6939, 20.0, 56.0, -38.621, 20.0, 1.0, 1.0, -1.0, -1.0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [7200.0, 6000.0, 7200.0, 4000.0, 1300.0, -4228.57, 210.204, 975.862, 500.0, 250.0], [1, 2, 7, 8], [7200.0, 7200.0, 1.0, 1.0], Any["L", "L", "L", "L", "L", "L", "E", "E"])

## Parsing `.tim` file

In [154]:
include("src/time_parser.jl")

parse_time_file (generic function with 1 method)

In [155]:
name, is_lp, col_periods, row_periods = parse_time_file("tssp/AIRL.tim")

("AIRL", false, ["X11", "X112"], ["HOURS1", "AVAIL11"])

### Extract A, T, W, c, q, b, h

In [156]:
col1, col2 = 
col2idx[col_periods[1]]:(col2idx[col_periods[2]]-1),  # First-period variables
col2idx[col_periods[2]]:d["ncols"]  # Second-period variables

(1:4, 5:12)

In [157]:
row1, row2 = 
row2idx[row_periods[1]]:(row2idx[row_periods[2]]-1),  # First-period rows
row2idx[row_periods[2]]:d["nrows"]  # Second-period variables

(1:2, 3:8)

In [195]:
# Right-hand sides and objectives
obj = sparsevec(objI, objV, d["ncols"])
c, q = Vector(obj[col1]), Vector(obj[col2])

([7200.0, 6000.0, 7200.0, 4000.0], [1300.0, -4228.57, 210.204, 975.862, 500.0, 250.0, 0.0, 0.0])

In [196]:
rhs = sparsevec(rhsI, rhsV, d["nrows"])
b, h = Vector(rhs[row1]), Vector(rhs[row2])

([7200.0, 7200.0], [0.0, 0.0, 0.0, 0.0, 1.0, 1.0])

In [197]:
A_ = sparse(I, J, V);

In [198]:
A = A_[row1, col1]
T = A_[row2, col1]
W = A_[row2, col2];

## Parsing `.sto` file

In [199]:
include("src/stoch_parser.jl")

parse_block_line! (generic function with 1 method)

In [239]:
name, indeps, blocks = parse_sto_file("tssp/AIRL.sto.first");

In [240]:
S_ = Base.Iterators.product(values(indeps)..., values(blocks)...)
S = [prod(collect(s)) for s in S_]
length(S)

25

In [222]:
function generate_scenario_data(
        A, T, W, q, h,
        row2idx, col2idx, 
        prob, coeffs
)
    m1, n1 = size(A)
    m2, n2 = size(W)
    m2 == size(T, 1) || error("T and W has different number of rows.")
    n1 == size(T, 2) || error("T and A has different number of columns.")
    
    # Create problem data
    h_ = copy(h)          # Right-hand side
    q_ = prob .* copy(q)  # Objective ()
    
    T_ = copy(T)          # T_r matrix
    W_ = copy(W)          # W_r
    
    
    for (k, v) in coeffs
        row::Int = row2idx[k[1]]
        col::Int = col2idx[k[2]]
        
        row == 0 || row > m1 || error("Entry given for first-stage row $(k[1])")
        
        # Check four cases
        if row == 0 && col > 0
            # Objective coefficient
#             println("Obj: $k: $v")
            q_[col - n1] = v
            
        elseif row > 0 && col == 0
            # Right-hand side
#             println("Rhs: $k \t $v")
            h_[row - m1] = v
            
        elseif row > 0 && col > 0
            # Check whether belongs to T or W
            if col <= n1
                T[row - m1, col] = v
            else
                W[row - m1, col - n1] = v
            end
        else
            eror("Wrong combination of row/col indices: $k")
        end
    end
    
    
    return T_, W_, q_, h_
end

generate_scenario_data (generic function with 2 methods)

In [241]:
pb_data = [generate_scenario_data(A, T, W, q, h, row2idx, col2idx, s.p, s.values) for s in S][:];

In [242]:
T_ = [x[1] for x in pb_data];
W_ = [x[2] for x in pb_data];
q_ = [x[3] for x in pb_data];
h_ = [x[4] for x in pb_data];

In [243]:
m1, n1 = size(A)
m2, n2 = size(W)
R = length(S)

25

In [244]:
rhs_ = vcat(b, h_...)
obj_ = vcat(c, q_...)
senses_ = vcat(senses[1:m1], repeat(senses[(m1+1):end], R))
n_ = length(obj_)
m_ = length(rhs_)
A_ = vcat(
    hcat(A, spzeros(m1, n2*R)),
    [
        hcat(
            T_[r],
            spzeros(m2, n2*(r-1)),
            W_[r],
            spzeros(m2, n2*(R-r))
        )
        for r in 1:R
    ]...
);

rhs_l = rhs_ .- Inf .* (senses_ .== "L");
rhs_u = rhs_ .+ Inf .* (senses_ .== "G");

In [258]:
nnz(W_[1]) / (prod(size(W_[1])))

0.3333333333333333

In [147]:
import MathProgBase
MPB = MathProgBase

MathProgBase

In [148]:
import CPLEX: CplexSolver

In [251]:
airl = MPB.LinearQuadraticModel(CplexSolver())
MPB.loadproblem!(airl, A_, zeros(n_), Inf*ones(n_), obj_, rhs_l, rhs_u, :Min)

In [252]:
MPB.optimize!(airl)

Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 50 columns.
Reduced LP has 152 rows, 154 columns, and 554 nonzeros.
Presolve time = 0.00 sec. (0.10 ticks)

Iteration log . . .
Iteration:     1   Scaled dual infeas =          4115.815549
Iteration:    62   Scaled dual infeas =           347.530648
Iteration:    76   Dual objective     =         37161.949063
Iteration:   137   Dual objective     =        246003.014510


0.003462076187133789

In [253]:
MPB.getobjval(airl)

249101.67207233087

### Build $T_{r}$, $W_{r}$

# Check for parsing errors

In [79]:
n_large = 0

for f in sto_files
    
    try
        name, indeps, blocks = parse_sto_file("tssp/"*f)
        

        # Check if some blocms must be compared to ref block
        for br in blocks
            realizations = br[2]

            ref = realizations[1].values

            for r_ in realizations[2:end]
                # check whether some keys are in ref but not in current block
                s_ = setdiff(keys(ref), keys(r_.values))
                length(s_) > 0 && println("\t", f, "\n", br[1])
            end
        end
        
        # Compute all scenarios
        S_ = Base.Iterators.product(values(indeps)..., values(blocks)...)
        S = [prod(collect(s)) for s in S_]
        length(S) >= 1000 && println(name, "\n\t|S| = $(length(S))")
        if length(S) >= 1000
            n_large += 1
        end
        
    catch err
        println(f, "\tError: $err")
    end
    
end

4NODECAR
	|S| = 1024
4NODECAR
	|S| = 16384
4NODECAR
	|S| = 2048
4NODECAR
	|S| = 32768
4NODECAR
	|S| = 4096
4NODECAR
	|S| = 8192
ASS2BY5
	|S| = 37500
ENV
	|S| = 1200
ENV
	|S| = 1875
ENV
	|S| = 32928
ENV
	|S| = 3780
ENV
	|S| = 5292
ENV
	|S| = 8232
fxmev.sto	Error: MethodError(Base.reduce_empty, (Base.mul_prod, Union{}), 0x000000000000620c)
PHONE
	|S| = 32768
stocfor1.sto	Error: MethodError(Base.reduce_empty, (Base.mul_prod, Union{}), 0x000000000000620c)
storm
	|S| = 1000
